# Random Forest Implementatoion

In [5]:
import pandas as pd
pd.set_option('display.max_columns', None)

df_train =  pd.read_csv('../../data/F_feature_selection/feature_selection-2022.csv', sep=';')
df_test = pd.df_test = pd.read_csv('../../data/F_feature_selection/feature_selection-2023.csv', sep=';')

In [6]:
df_test.head()


,location,type_of_collision,latitude,longitude,reserved_lane_present,horizontal_alignment,infrastructure,accident_situation,speed_limit,position,sex,pedestrian_location,pedestrian_action,fixed_obstacle_struck,mobile_obstacle_struck,initial_point_of_impact,main_maneuver_before_accident,motor_type,fixed_obstacle_struck_other,mobile_obstacle_struck_other,initial_point_of_impact_other,main_maneuver_before_accident_other,motor_type_other,vehicle_category_involved_bicycle,vehicle_category_involved_bus_coach,vehicle_category_involved_hgv_truck,vehicle_category_involved_light_motor_vehicle,vehicle_category_involved_other,vehicle_category_involved_powered_2_3_wheeler,hour_sin,hour_cos,time_of_day,day_of_week,day_of_week_sin,day_of_week_cos,month_sin,month_cos,day_of_year_sin,day_of_year_cos,age,age_group,used_belt,used_helmet,used_child_restraint,used_airbag,vehicle_category_simplified,impact_score,vehicle_category_simplified_other,impact_score_other,impact_delta,road_complexity_index,surface_quality_indicator,role,lighting_ordinal,weather_ordinal,injury_target
0,2,7,48.866386,2.323471,0,1,0,1,30,1,1,-1,-1,0,0,5,1,1,-1,-1,-1,-1,-1,0,0,0,0,0,1,1.000000,6.123234e-17,Morning_Rush,6,-0.781831,0.62349,0.5,-0.866025,0.817401,-0.576069,45,middle_aged,0,1,0,0,powered_2_3_wheeler,3,none,1,2,3.750000,0,driver,0,2,1
1,2,6,48.845478,2.428681,2,1,0,1,50,1,2,-1,-1,0,1,1,1,1,-1,-1,-1,-1,-1,0,0,0,1,0,0,0.965926,2.588190e-01,Night,6,-0.781831,0.62349,0.5,-0.866025,0.817401,-0.576069,26,adult,1,0,0,0,light_motor_vehicle,4,none,1,3,4.166667,0,driver,1,3,0
2,2,6,48.845478,2.428681,2,1,0,1,50,10,1,2,3,-1,-1,-1,-1,-1,0,1,1,1,1,0,0,0,1,0,0,0.965926,2.588190e-01,Night,6,-0.781831,0.62349,0.5,-0.866025,0.817401,-0.576069,26,adult,0,0,0,0,unknown,1,light_motor_vehicle,4,-3,4.166667,0,pedestrian,1,3,2
3,2,1,48.762400,2.406550,0,1,5,1,50,1,1,0,0,0,2,1,16,1,0,2,2,15,1,0,0,0,1,0,1,-0.866025,5.000000e-01,Night,6,-0.781831,0.62349,0.5,-0.866025,0.817401,-0.576069,36,adult,0,1,0,0,powered_2_3_wheeler,3,light_motor_vehicle,4,-1,5.833333,0,driver,0,2,2
4,2,1,48.762400,2.406550,0,1,5,1,50,1,2,0,0,0,2,2,15,1,0,2,1,16,1,0,0,0,1,0,1,-0.866025,5.000000e-01,Night,6,-0.781831,0.62349,0.5,-0.866025,0.817401,-0.576069,39,adult,1,0,0,0,light_motor_vehicle,4,powered_2_3_wheeler,3,1,5.833333,0,driver,0,2,0


In [9]:
columns_to_drop = [
    'latitude', 'longitude',
    'pedestrian_location', 'pedestrian_action',
    'hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos', 'day_of_year_sin', 'day_of_year_cos', 'age',
]

object_type = ['role', 'age_group', 'vehicle_category_simplified', 'vehicle_category_simplified_other', 'time_of_day']

na_with_meaning = [
    'fixed_obstacle_struck', 'mobile_obstacle_struck', 'initial_point_of_impact', 'main_maneuver_before_accident', 'motor_type',
    'fixed_obstacle_struck_other', 'mobile_obstacle_struck_other', 'initial_point_of_impact_other', 'main_maneuver_before_accident_other', 'motor_type_other',
    'sex', 
    
    #'pedestrian_location', 'pedestrian_action'
]

columns_to_int32 = [
    'location', 'type_of_collision', 'reserved_lane_present', 'horizontal_alignment', 'infrastructure', 'accident_situation', 'position',
    'fixed_obstacle_struck', 'mobile_obstacle_struck', 'initial_point_of_impact', 'main_maneuver_before_accident', 'motor_type', 
    'fixed_obstacle_struck_other', 'mobile_obstacle_struck_other', 'initial_point_of_impact_other', 'main_maneuver_before_accident_other',
    'motor_type_other','vehicle_category_involved_bicycle', 'vehicle_category_involved_bus_coach', 'vehicle_category_involved_hgv_truck',
    'vehicle_category_involved_light_motor_vehicle', 'vehicle_category_involved_other', 'vehicle_category_involved_powered_2_3_wheeler',
    'used_belt', 'used_helmet', 'used_child_restraint', 'used_airbag', 'impact_score', 'impact_score_other', 'impact_delta', 'surface_quality_indicator',
    'lighting_ordinal', 'weather_ordinal', 'injury_target', 'sex', 'day_of_week', 'speed_limit'
]

In [10]:
for x in [df_train, df_test]:
    # Remove rows or columns with missing values that do not encode info.
    x.drop(columns=columns_to_drop, inplace=True)    
    x.replace(-1, pd.NA, inplace=True)
    x[na_with_meaning] = x[na_with_meaning].fillna(-1)
    x[['vehicle_category_simplified', 'vehicle_category_simplified_other']] = x[['vehicle_category_simplified', 'vehicle_category_simplified_other']].fillna('no vehicle')
    x.dropna(inplace=True)
    x[columns_to_int32] = x[columns_to_int32].astype('int32')
    
    # Drop rows with impossible speed limit
    # France has a maximum speed limit of 130
    x.drop(index=x[x['speed_limit'] > 130].index, inplace=True)

C:\Users\Aaron Niemesch\AppData\Local\Temp\ipykernel_24080\1167868644.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x[na_with_meaning] = x[na_with_meaning].fillna(-1)
C:\Users\Aaron Niemesch\AppData\Local\Temp\ipykernel_24080\1167868644.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x[na_with_meaning] = x[na_with_meaning].fillna(-1)


In [11]:
num_cols = ['speed_limit', 'day_of_week', 'impact_score', 'impact_score_other', 'impact_delta', 'road_complexity_index', 'surface_quality_indicator', 'lighting_ordinal', 'weather_ordinal', 'vehicle_category_involved_bicycle', 'vehicle_category_involved_bus_coach', 'vehicle_category_involved_hgv_truck','vehicle_category_involved_light_motor_vehicle', 'vehicle_category_involved_other', 'vehicle_category_involved_powered_2_3_wheeler']

cat_cols = df_train.columns.difference(num_cols + ['injury_target'])

In [14]:
X_train = df_train.drop(columns='injury_target')
y_train = df_train['injury_target']

X_test = df_test.drop(columns='injury_target')
y_test = df_test['injury_target']

In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 108527 entries, 1 to 126243
Data columns (total 43 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   location                                       108527 non-null  int32  
 1   type_of_collision                              108527 non-null  int32  
 2   reserved_lane_present                          108527 non-null  int32  
 3   horizontal_alignment                           108527 non-null  int32  
 4   infrastructure                                 108527 non-null  int32  
 5   accident_situation                             108527 non-null  int32  
 6   speed_limit                                    108527 non-null  int32  
 7   position                                       108527 non-null  int32  
 8   sex                                            108527 non-null  int32  
 9   fixed_obstacle_struck                     